In [1]:
# CSC1142 - Cloud Technologies Assignment
# Global Energy Transition Analytics Pipeline
# STUDENT NAME: Om Nilesh Bharti
# STUDENT ID: A00049459
# EMAIL ID: omnilesh.bharti2@mail.dcu.ie



# Step 1: Environment Setup
!pip install pyspark -q

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, avg, round, desc
from pyspark import SparkFiles

# Initialize the Spark Session
spark = SparkSession.builder \
    .appName("Green_Energy_Analytics_Pipeline") \
    .getOrCreate()

print(" Spark Session Initialized")



# Step 2: Data Ingestion (ETL - Extract)
# Programmatically ingest raw CSV data from the remote GitHub repository
url = "https://raw.githubusercontent.com/owid/energy-data/master/owid-energy-data.csv"
spark.sparkContext.addFile(url)

# Load data into a Spark DataFrame with schema inference
df = spark.read.csv(SparkFiles.get("owid-energy-data.csv"), header=True, inferSchema=True)

print(f" Data Ingested. Total Rows: {df.count()}")

# Step 3: Data Cleaning (ETL - Transform)
# Filter criteria:
# 1. Year >= 2000
# 2. iso_code IS NOT NULL
# 3. gdp IS NOT NULL
df_clean = df.filter(
    (col("year") >= 2000) &
    (col("iso_code").isNotNull()) &
    (col("gdp").isNotNull())
)


# Step 4: Feature Engineering
df_filled = df_clean.fillna(0, subset=["solar_consumption", "wind_consumption", "hydro_consumption", "coal_consumption", "oil_consumption"])

df_transformed = df_filled \
    .withColumn("Total_Green", col("solar_consumption") + col("wind_consumption") + col("hydro_consumption")) \
    .withColumn("Total_Fossil", col("coal_consumption") + col("oil_consumption"))


# Step 5: Aggregation & Analysis
final_analysis = df_transformed.groupBy("country") \
    .agg(
        round(avg("Total_Green"), 2).alias("Avg_Green_Energy_TWh"),
        round(avg("Total_Fossil"), 2).alias("Avg_Fossil_Energy_TWh"),
        round(avg("gdp"), 0).alias("Avg_GDP")
    ) \
    .withColumn("Green_Ratio", round(col("Avg_Green_Energy_TWh") / (col("Avg_Green_Energy_TWh") + col("Avg_Fossil_Energy_TWh")), 4)) \
    .filter(col("Avg_Green_Energy_TWh") > 10) # Filter out micro-nations with negligible data

# Sort by the calculated Green Ratio to identify top performers
result = final_analysis.orderBy(desc("Green_Ratio"))

# Display the Top 15 countries
print("---  Final Output: Top 15 Countries by Green Energy Ratio ---")
result.show(15)

# Step 6: Data Loading (ETL - Load)
result.write.mode("overwrite").csv("/content/Green_Energy_Report", header=True)
print(" Pipeline Completed. Results saved to /content/Green_Energy_Report")

 Spark Session Initialized
 Data Ingested. Total Rows: 23195
---  Final Output: Top 15 Countries by Green Energy Ratio ---
+-----------+--------------------+---------------------+-----------------+-----------+
|    country|Avg_Green_Energy_TWh|Avg_Fossil_Energy_TWh|          Avg_GDP|Green_Ratio|
+-----------+--------------------+---------------------+-----------------+-----------+
|     Norway|              345.81|               122.82| 3.79320984752E11|     0.7379|
|    Iceland|               28.48|                10.84|  1.2516309219E10|     0.7243|
|     Sweden|              200.86|               199.83|  3.9787892036E11|     0.5013|
|     Brazil|             1001.44|              1442.32|2.631366220817E12|     0.4098|
|Switzerland|               94.67|               136.91| 4.48234996607E11|     0.4088|
|     Canada|             1002.23|              1496.81| 1.45457781879E12|      0.401|
|New Zealand|                67.3|               104.47| 1.48639456292E11|     0.3918|
|    Au